In [70]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, Dropout, Conv1D, ReLU, BatchNormalization,Add, AveragePooling1D, Flatten, Dense
from tensorflow.keras.losses import mean_squared_error, binary_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow import Tensor

In [81]:
def relu_bn(inputs: Tensor) -> Tensor:
    relu = ReLU()(inputs)
    bn = BatchNormalization()(relu)
    return bn

def residual_block(x: Tensor, downsample: bool, filters: int, kernel_size: int = 3) -> Tensor:
    y = Conv1D(kernel_size=kernel_size,
               strides= (1 if not downsample else 2),
               filters=filters,
               padding="same")(x)
    y = relu_bn(y)
    y = Dropout(.2)(y)
    y = Conv1D(kernel_size=kernel_size,
               strides=1,
               filters=filters,
               padding="same")(y)

    if downsample:
        x = Conv1D(kernel_size=1,
                   strides=2,
                   filters=filters,
                   padding="same")(x)
    out = Add()([x, y])
    out = relu_bn(out)
    out = Dropout(.2)(out)
    return out

def create_res_net(input_dim,num_filters):
    
    inputs = Input(input_dim)
    num_filters = num_filters
    
    t = BatchNormalization()(inputs)
    t = Conv1D(kernel_size=2,
               strides=1,
               filters=num_filters,
               padding="same")(t)
    t = relu_bn(t)
    t = Dropout(.2)(t)
    
    num_blocks_list = [2, 5, 5, 2]
    for i in range(len(num_blocks_list)):
        num_blocks = num_blocks_list[i]
        for j in range(num_blocks):
            t = residual_block(t, downsample=(j==0 and i!=0), filters=num_filters)
        num_filters *= 2
    
    t = AveragePooling1D(3)(t)
    t = Flatten()(t)
    outputs = Dense(10, activation='softmax')(t)
    
    model = Model(inputs, outputs)

    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model    

In [71]:
# read in data
df = pd.read_csv("liverData/eldd.csv")

In [72]:
# print length and show sample data
print(len(df))
df.head()

638


44

In [73]:
#meld functions
#https://www.mdcalc.com/calc/10437/model-end-stage-liver-disease-meld#evidence

import math

# TODO: check all SI values and see what needs conversion. Sodium did not, unknown after Bili but placeholder functions added
#convert SI units to US units
#creatinine umol/L to mg/dl
def convertCreatinine(CRE_S):
     return (CRE_S * 0.0230)
#bilirubin umol/L to mg/dl
def convertBilirubin(BILI_S):
     return (BILI_S * 0.0585)
#bilirubin umol/L to mg/dl
def convertCystatinC(BILI_S):
     return (BILI_S * 0.0585)
#bilirubin umol/L to mg/dl
def convertIL_6(BILI_S):
     return (BILI_S * 0.0585)
#bilirubin umol/L to mg/dl
def convertWBC(BILI_S):
     return (BILI_S * 0.0585)
#bilirubin umol/L to mg/dl
def convertProtien(BILI_S):
     return (BILI_S * 0.0585)
def convertAlbumin(BILI_S):
     return (BILI_S * 0.0585)
def convertALAT(BILI_S):
     return (BILI_S * 0.0585)
def convertASAT(BILI_S):
     return (BILI_S * 0.0585)

#meld3
#1.33*(Female) + 4.56*ln(Serum bilirubin) + 0.82*(137 - Sodium) – 0.24*(137 - Sodium)*ln(Serum bilirubin) + 9.09*ln(INR) + 11.14*ln(Serum creatinine) + 1.85*(3.5 – Serum albumin) – 1.83*(3.5 – Serum albumin)*ln(Serum creatinine) + 6,
#rounded to the nearest integer
#Serum bilirubin, INR, and serum creatinine values below 1.0 are set to 1.0.
#Sodium is limited to a range of 125-137 mEq/L, and if outside of these bounds, is set to the nearest limit.
#Serum albumin is limited to a range of 1.5-3.5 g/dL, and if outside of these bounds, is set to the nearest limit.
#Maximum serum creatinine is 3.0 mg/dL, and if above this bound, is set to 3.0 mg/dL.

def calm_meld(CRE_S, BILI_S, INR_C):
    meld_score = (0.957*math.log(convertCreatinine(CRE_S)) + 0.378*math.log(convertBilirubin(BILI_S)) + 1.120*math.log(INR_C) + 0.643)*10
    return int(round(meld_score))

def calc_meld3(CRE_S, BILI_S, INR_C, NA_S, ALB_S, sex):
    BILI_S = convertBilirubin(BILI_S)
    CRE_S = convertCreatinine(CRE_S)
    if BILI_S < 1:
        BILI_S = 1
    if INR_C < 1:
        INR_C = 1
    if CRE_S < 1:
        CRE_S = 1
    if CRE_S >= 3:
        CRE_S = 3
    if NA_S > 137:
        NA_S = 137
    if NA_S < 125:
        NA_S = 125
    if ALB_S > 3.5:
        ALB_S = 3.5
    if ALB_S < 1.5:
        ALB_S = 1.5
    if sex == 'female':
        sex = 1
    else:
        sex = 0
    
    meld3_score = 1.33*(sex) + 4.56*math.log(BILI_S) + 0.82*(137 - NA_S) - 0.24*(137 - NA_S)*math.log(BILI_S) + 9.09*math.log(INR_C) + 11.14*math.log(CRE_S) + 1.85*(3.5 - ALB_S) - 1.83*(3.5 - ALB_S)*math.log(CRE_S) + 6
    return int(round(meld3_score))

#meld_na
#MELD(i) = 0.957*ln(Creatinine) + 0.378*ln(Bilirubin) + 1.120*ln(INR) + 0.643
#Then, round to the tenth decimal place and multiply by 10. 
#If MELD(i) > 11, perform additional MELD calculation as follows:
#MELD = MELD(i) + 1.32*(137 – Na) –  [0.033*MELD(i)*(137 – Na)]

def calc_meldNA(CRE_S, BILI_S, INR_C,NA_S):
    meldNA_score = 0.957 * math.log(convertCreatinine(CRE_S)) + 0.378*math.log(convertBilirubin(BILI_S)) + 1.120*math.log(INR_C) + 0.643
    meldNA_score = round(meldNA_score, 1) * 10
    if meldNA_score > 11:
        meldNA_score = meldNA_score + 1.32*(137-NA_S) - (0.033*meldNA_score*(137-NA_S))
    return int(round(meldNA_score))


In [74]:
#Data preprocessing
#drop NA rows to allow for meld scores. 654 down to 638
df = df.dropna(subset=['INR_C','NA_S','ALB_S'])

#TODO: convert SI to metric as necessary
#US hospital labs are in metric is why we're converting here.


In [75]:
df2 = df.copy(deep=True)
df2['MELD'] = df2.apply(lambda row: calm_meld(row["CRE_S"],row["BILI_S"],row["INR_C"]), axis =1)
df2['MELD3'] = df2.apply(lambda row: calc_meld3(row["CRE_S"],row["BILI_S"],row["INR_C"],row["NA_S"],row["ALB_S"],row["Sex"]), axis =1)
df2['MELDNA'] = df2.apply(lambda row: calc_meldNA(row["CRE_S"],row["BILI_S"],row["INR_C"],row["NA_S"]), axis =1)


In [76]:
df2.head()

,Age,Sex,DaysAtRisk,Deceased,LTx,Cirrhosis,ALF,Ethyltoxic,HBV,HCV,...,INR_C,NA_S,P_S,PALB_S,PROT_S,PTH_S,VDT_OH_S,MELD,MELD3,MELDNA
0,68,male,200,0,0,1.0,0,1,1,0,...,1.11,135.4,1.49,0.19,69.6,2.39,12.7,13,18,14
1,64,male,3,1,0,1.0,0,1,0,0,...,1.77,133.1,0.96,0.05,62.5,19.39,5.5,35,30,36
2,67,female,208,0,0,1.0,0,1,0,0,...,1.09,137.4,1.14,0.17,80.5,7.39,18.8,15,17,15
3,32,female,17,1,0,0.0,1,0,0,0,...,2.29,130.5,1.71,0.15,48.9,2.21,4.5,29,32,31
4,64,female,189,0,0,1.0,0,1,0,0,...,1.10,142.6,1.07,0.11,67.6,4.17,34.1,15,18,10


In [85]:
from sklearn.model_selection import train_test_split

#Trim columns to use for training
#Removing days at risk, deceased, 
#fields = ['Age','Sex','Cirrhosis','ALF','Ethyltoxic','HBV','HCV','INR_C','NA_S','P_S','','','','']
x = df2.copy(deep=True)

#Goal of improving on original meld for now. 
#Meld3 is used primarily for transplant, meldNA is used for Cirrhosis: #https://www.mdcalc.com/calc/10437/model-end-stage-liver-disease-meld#evidence
y = df2['MELD'].copy(deep=True)

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=7)

In [88]:
model = create_res_net()

TypeError: create_res_net() missing 2 required positional arguments: 'input_dim' and 'num_filters'

In [ ]:
#TODO: custom accuracy metric. Not trying to match current meld value, but to prove our model is 
#      generating a better measure of short term survivability. 